In [ ]:
from sklearn import svm
# from category_encoders import *
import pandas as pd
from sklearn.datasets import load_boston
import pandas as pd
import numpy as np
from utils import *
# from MLP import *
import networkx as nx
import xgboost as xgb
import sys
sys.path.append('../data/pengpai/')
import address
from bidict import bidict
from sklearn.metrics import accuracy_score
# from sklearn.preprocessing import LabelEncoder
import pandas as pd
from dataloader import get_dataloader
import argparse

from torch.nn.utils.rnn import pad_sequence
HEAD = 0
TAIL = 1

In [ ]:
parser = argparse.ArgumentParser()
parser.add_argument('--model_name', type=str, default='att_model', help='model name ')
parser.add_argument('--lr_LP', type=float, default=18e-5, help='learning rate')
# parser.add_argument('--lr_path', type=float, default=1e-4, help='learning rate')
parser.add_argument('--ratio', type=float, default=1.0, help='ratio of neg samples and pos samples')
parser.add_argument('--num_epochs', type=int, default=100, help='number of epochs')
parser.add_argument("--batch_size", type=int, default=128, help="LP batch size")
parser.add_argument("--model_save_dir", type=str, default="saved/model/MLP", help="save dir for model")
parser.add_argument("--HEAD", type=int, default=0, help="")
parser.add_argument("--TAIL", type=int, default=1, help="")
parser.add_argument("--LABEL", type=int, default=2, help="")
parser.add_argument("--upperboundratio_10time", type=int, default=2,
                    help="upperboundratio_10time for ratio analysis")
parser.add_argument("--sample_type", type=str, default='random',
                    help="sample type for pos and neg case pairs")
parser.add_argument("--gpu", type=int, default=2, help="gpu device")
parser.add_argument("--proportion", type=float, default=1.0, help="proportion of the total data for train and test")
# parser.add_argument("--importance_threshold", type=float, default=0.2, help="place importance threshold")
# parser.add_argument("--classification_num_epoch", type=int, default=100, help="number for classification epochs")
# parser.add_argument("--classification_lr", type=float, default=1e-4, help='classification learning rate')
parser.add_argument("--run_model", type=str, default='ablation')
# parser.add_argument("--load_epo",type=int,default=-1)
parser.add_argument("--lbd", type = int,default=4)
parser.add_argument("--get_best", type=bool, default=False)
parser.add_argument("--epo", type=int, default=-1)
parser.add_argument('--path_len',type = int, default= 45)
parser.add_argument("--num_gb_epoch",type=int,default=20,help="number of epoch for geting best model")
parser.add_argument('--rl_name',type = str,default='hprl_carl')
parser.add_argument('--baseline_save_path',type = str,default = 'saved/baselines')
# parser.add_argument("--")
# parser.add_argument("--lbd_sen",type=bool,default=False,help = "Sensitivity analysis for lbd")
config = parser.parse_args(args=[])
    

In [ ]:
# reindexed_sp_net, old2new_case_id, reindexed_case_tensor_padded = get_case_tensor('gne')
# train_dataloader, test_dataloader, all_dataloader = get_dataloader(config, reindexed_sp_net, old2new_case_id)

In [ ]:
# case_idx_pair_train = train_dataloader.dataset.dataset.data
# case_idx_pair_test = test_dataloader.dataset.dataset.data
# case_idx_pair_train = load_file('dataset/case_idx_pair/random/train_4.pickle')
# case_idx_pair_test = load_file('dataset/case_idx_pair/random/test_4.pickle')

In [ ]:
# case_idx_pair_train = np.array(train_dataloader.dataset.dataset)[train_dataloader.dataset.indices]
# case_idx_pair_test = np.array(train_dataloader.dataset.dataset)[test_dataloader.dataset.indices]

In [ ]:
# case_idx_pair_train.dataset.data
def get_train_test_np(case_idx_pair,reindexed_case_tensor_padded):
    case_pair_tensor=[]
    for (case_i,case_j,lab) in case_idx_pair:
        emb = []
        emb += reindexed_case_tensor_padded[case_i].tolist()
        emb += reindexed_case_tensor_padded[case_j].tolist()
        emb += [lab]
        case_pair_tensor+=[emb]
    return np.array(case_pair_tensor)

In [ ]:
# case_tensor_pair_train = get_train_test_np(case_idx_pair_train,reindexed_case_tensor_padded)
# case_tensor_pair_test = get_train_test_np(case_idx_pair_test,reindexed_case_tensor_padded)


In [ ]:
# case_tensor_pair_train.shape == case_tensor_pair_test.shape

In [ ]:
# case_tensor_pair_train.shape

In [ ]:
from sklearn.utils import shuffle
def get_train_test(model_name):
    reindexed_sp_net, old2new_case_id, reindexed_case_tensor_padded = get_case_tensor(model_name)
    train_dataloader, test_dataloader, all_dataloader = get_dataloader(config, reindexed_sp_net, old2new_case_id)
    case_idx_pair_train = load_file('dataset/case_idx_pair/random/train_4.pickle')
    case_idx_pair_test = load_file('dataset/case_idx_pair/random/test_4.pickle')
    case_tensor_pair_train = get_train_test_np(case_idx_pair_train,reindexed_case_tensor_padded)
    case_tensor_pair_test = get_train_test_np(case_idx_pair_test,reindexed_case_tensor_padded)
    tensor_train = shuffle(case_tensor_pair_train)
    X_train = tensor_train[:,:-1]
    y_train = tensor_train[:,-1]

    tensor_test = shuffle(case_tensor_pair_test)
    X_test = tensor_test[:,:-1]
    y_test = tensor_test[:,-1]
    return X_train,y_train,X_test,y_test

In [ ]:
def train_test(model_name):
    
    X_train,y_train,X_test,y_test = get_train_test(model_name)
    model = xgb.XGBClassifier()
    model.fit(X_train,y_train)
    y_test,y_pred =  y_test, model.predict(X_test)
    
    accuracy = metrics.accuracy_score(y_test, y_pred)
    print("Accuracy : %.4g" % accuracy) 
    precision = metrics.precision_score(y_test,y_pred)
    print('Precision: %.4g' % precision)
    recall = metrics.recall_score(y_test,y_pred)
    print('Recall: %.4g' % metrics.recall_score(y_test,y_pred))
    f1 = metrics.f1_score(y_test,y_pred)
    print('F1: %.4g' % f1)

    y_proba = model.predict_proba(X_test)[:,1]
    auc = metrics.roc_auc_score(y_test,y_pred)
    print("AUC Score (Test): %.4g" % metrics.roc_auc_score(y_test, y_pred))
    return (y_test,y_pred,accuracy,precision,recall,f1,auc)
    

In [ ]:
print('model_name,accuracy,precision,recall,f1,auc')
model_list = ['hprl_carl']
res = load_file('saved/results/XGBoost.pickle')
for model_name in model_list:
    print('train on:{}'.format(model_name))
    res[model_name] = train_test(model_name)
    save_pickle(obj = res, filepath='saved/results/XGBoost.pickle')

In [ ]:
res['hprl_carl']